In [15]:
from tensorflow import keras
from PIL import Image
from torchvision import models, transforms
import torch.nn as nn
import torch.optim as optim
import torch
import random
from torchvision import models, transforms
from numpy.linalg import norm
import numpy as np
import math

# Load Data

In [16]:
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
import torch

transform = transforms.Compose([
    transforms.Resize((224, 244)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), #Is transformed to be better with VGG16 model
    ])

In [17]:
def getDataset(train=True, sample_size=100):
    dataset = CIFAR10(root='./data',
                  train=train, 
                  download=True,
                  transform=transform)
    
    X = []
    y = []

    for i in range(sample_size):
        X.append( dataset[i][0] )
        y.append( dataset[i][1] )

    X = torch.stack( X )

    return X, y

X_train, y_train = getDataset() #Training data
X_test, y_test = getDataset(train=False) #Test data

Files already downloaded and verified
Files already downloaded and verified


# Create Minibatch

In [18]:
batchSize = 32

In [19]:
indices = np.random.choice(X_train.shape[0], batchSize, replace=False) # Not used as of now
minibatch = X_train[indices]
#minibatch

# Load Model

In [20]:
from torchvision import models
pretrained_model = models.vgg16(pretrained=True)
pretrained_model.classifier = torch.nn.Identity() # 25088 features output

c:\Users\Test\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Test\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [21]:
class CustomNN(nn.Module):
    def __init__(self):
        super(CustomNN, self).__init__()
        self.fc_layers = nn.Sequential(
            nn.Linear(25088, 10000),  # First fully connected layer
            nn.ReLU(),
            nn.Linear(10000, 4000)    # Second fully connected layer to reduce to 4000
        )

    def forward(self, x):
        return self.fc_layers(x)

In [22]:
class CombinedModel(nn.Module):
    def __init__(self, cnn_extractor, custom_nn):
        super(CombinedModel, self).__init__()
        self.cnn_extractor = cnn_extractor  # Pretrained CNN (frozen)
        self.custom_nn = custom_nn  # Custom NN (trainable)

    def forward(self, x):
        features = self.cnn_extractor(x)  # Extract features using the CNN
        output = self.custom_nn(features)  # Pass the features through the custom NN
        return output


In [23]:
custom_nn = CustomNN()
model = CombinedModel(pretrained_model, custom_nn)

# Forward Propagate Through Model

In [24]:
result = model(X_train) # torch.Size([100, 4000])

In [25]:
# Parameters
mean = 0
std_dev = np.sqrt(0.01)  # Standard deviation is the square root of the variance

# Create the matrix W and V with shape (4000, 32) and (32, 1) - SHOULD BE A FUNCTION WHERE 32 IS THE NUMBER OF BITS WE WANT
W = torch.normal(mean, std_dev, (4000, 32))
V = torch.normal(mean, std_dev, (32, 1)) 

print(W.shape)
print(V.shape)

torch.Size([4000, 32])
torch.Size([32, 1])


In [26]:
U = []
for i in range(len(result)):
    dot = torch.matmul(W.T, result[i])
    dot = dot.reshape(32, 1)
    u = (dot + V)
    U.append(u)
U = torch.stack(U) # torch.Size([100, 32, 1])
U = U.reshape(100, 32)

In [35]:
# Compute the dot product matrix -> Useful for finding Theta in loss function
dot_product_matrix = torch.matmul(U, U.T)
dot_product_matrix #(100, 100) Shape -> (sample_size, sample_size) Shape

#Calculate Theta
Theta = 1/2 * dot_product_matrix
Theta.shape

torch.Size([100, 100])

# Finding S Based On Labels

In [28]:
y_train = torch.tensor(y_train) # (sample_size)
S = (y_train[:, None] == y_train).float() # (sample_size, sample_size) Shape -> 1 if labels are the same, 0 if not

# Finding binary codes B

In [29]:
#Find the Binary codes
B = torch.sign(U)

## Custom Loss Function

In [30]:
#Parameters
eta = 0.25 #Learning Rate

In [31]:
S

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 1.,  ..., 0., 0., 0.],
        [0., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 1.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 1., 0., 1.]])

In [32]:
Theta

tensor([[ 9.6752,  8.6843,  5.0221,  ...,  5.7804,  3.1493,  7.0362],
        [ 8.6843, 16.8660,  8.2876,  ...,  9.6693,  4.9741, 11.9777],
        [ 5.0221,  8.2876,  9.9056,  ...,  7.3979,  2.4237,  7.0048],
        ...,
        [ 5.7804,  9.6693,  7.3979,  ..., 12.0618,  3.5190, 10.9604],
        [ 3.1493,  4.9741,  2.4237,  ...,  3.5190,  7.1585,  5.3174],
        [ 7.0362, 11.9777,  7.0048,  ..., 10.9604,  5.3174, 21.2790]],
       grad_fn=<MulBackward0>)

In [38]:
Theta.shape

torch.Size([100, 100])

In [57]:
U.shape

torch.Size([100, 32])

In [51]:
for u in U:
    for i in u:
        print(str(float(i))[:4], end = " ")
    print()

-0.0 -0.0 0.21 2.32 0.30 1.33 -0.4 1.86 0.17 -0.1 -0.4 0.50 -0.0 0.28 -0.2 -0.4 1.65 -0.0 -0.2 -1.1 0.73 0.18 -0.4 0.24 0.94 -0.5 0.07 -0.2 -0.9 0.05 0.45 0.03 
0.24 0.72 -0.9 2.27 0.96 1.50 -1.1 2.24 0.32 1.10 -0.0 0.28 1.47 0.84 -0.2 -0.6 0.67 0.29 -0.7 0.57 0.06 0.51 -0.0 -0.9 0.44 -1.9 0.34 -0.5 -2.1 -0.6 0.75 -0.0 
0.38 -0.0 -0.6 0.76 0.00 1.32 0.33 1.05 -0.9 -0.2 -0.2 -1.1 1.36 -0.0 0.01 0.01 0.91 0.57 -0.7 -0.1 -0.1 0.16 -0.3 -0.0 0.75 -2.4 0.55 0.16 -1.1 0.71 0.75 -0.6 
0.48 0.68 -0.3 1.49 1.08 -0.0 0.27 0.66 -0.3 0.72 0.29 -0.1 0.36 -0.2 -0.7 -0.8 0.50 0.10 -0.9 0.22 -0.0 -0.6 0.08 -0.6 0.34 -1.0 0.06 -1.3 -1.9 0.38 0.41 0.86 
1.27 0.13 -0.3 0.67 1.19 1.18 -0.5 2.02 0.29 -0.1 0.46 -0.4 1.03 0.43 -0.1 0.16 2.18 0.12 -0.2 0.55 -0.0 0.60 -0.3 0.52 1.42 -1.0 0.73 0.29 0.11 0.51 0.42 -0.8 
2.30 0.03 0.33 0.44 -0.4 3.67 0.38 0.62 -0.6 0.30 -0.3 -0.2 0.33 0.28 0.41 -0.3 -0.2 0.41 -0.1 0.92 0.17 0.50 1.56 0.03 -0.1 -1.9 0.64 -1.5 -2.4 0.33 1.11 1.34 
0.92 -0.1 0.48 1.39 1.26 2.03 0.41

In [ ]:
# Compute the dot product matrix -> Useful for finding Theta in loss function
dot_product_matrix = torch.matmul(U, U.T)
dot_product_matrix #(100, 100) Shape -> (sample_size, sample_size) Shape

#Calculate Theta
Theta = 1/2 * dot_product_matrix
Theta.shape

In [60]:
torch.sum(S)

tensor(1130.)

In [52]:
(S * Theta)

tensor([[ 9.6752,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000, 16.8660,  8.2876,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  8.2876,  9.9056,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ..., 12.0618,  0.0000, 10.9604],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  7.1585,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ..., 10.9604,  0.0000, 21.2790]],
       grad_fn=<MulBackward0>)

In [36]:
torch.sum(S * Theta)

tensor(8399.2637, grad_fn=<SumBackward0>)

In [12]:
import torch.nn as nn

class CustomLoss(nn.Module):
    def __init__(self, mean, std_dev, eta = 0.25, hash_length = 32):
        super(CustomLoss, self).__init__()
        # Initialize W and V as learnable parameters
        self.W = nn.Parameter(torch.normal(mean, std_dev, (4000, hash_length)))  # W will be updated during training
        self.V = nn.Parameter(torch.normal(mean, std_dev, (hash_length, 1)))     # V will be updated during training
        self.eta = eta  # Regularization parameter

    def forward(self, inputs, targets, hash_length = 32):
        targets = torch.tensor(targets)
        S = (targets[:, None] == targets).float() # S calculation
        U = (torch.matmul(self.W.T, inputs) + self.V).reshape(100, hash_length) # U calculation

        #Calculate Theta
        dot_product_matrix = torch.matmul(U, U.T)
        dot_product_matrix # (sample_size, sample_size) Shape
        Theta = 1/2 * dot_product_matrix

        #Calculate hash codes
        B = torch.sign(U)

        loss = - torch.sum(S * Theta - torch.log(1 + torch.exp(Theta)))
        loss += + self.eta * torch.sum(torch.norm(B - U, dim = 1).pow(2))    
        return loss.mean()


In [72]:
Theta2 = Theta * 10

In [73]:
torch.sum(torch.log( 1 + torch.exp(Theta2)))

tensor(inf, grad_fn=<SumBackward0>)

In [70]:
-8399 + 62676

54277

In [74]:
- torch.sum(S * Theta2 - torch.log(1 + torch.exp(Theta2)))

tensor(inf, grad_fn=<NegBackward0>)

# Using Custom Loss Function

In [13]:
num_epochs = 10

In [14]:
loss_fn = CustomLoss(mean=0, std_dev=0.01, eta=0.25)


for param in pretrained_model.parameters(): #Lås parameters i VGG16
    param.requires_grad = False

optimizer = optim.Adam(model.custom_nn.parameters(), lr=0.001)


for epoch in range(num_epochs):
    optimizer.zero_grad()  # Clear previous gradients
    inputs = X_train  # Your input data
    targets = y_train  # Your target labels
    outputs = model(inputs)  # Forward pass through the combined model
    loss = loss_fn(outputs, targets)  # Compute the custom loss
    loss.backward()  # Backpropagate
    optimizer.step()  # Update only the custom NN's parameters

NameError: name 'W' is not defined